# Stage 1 - Code Study Allennlp + DataReader, SparseAdjacencyField
- 8/?-8/?
- code study allennlp (across stages)
    - study tools and resources
        - allennlp guide
        - allennlp doc
        - allennlp github (source code)
        - google
    - data
        - fields
        - instances
        - batch
        - dataset
        - vocabulary
    - data operator
        - reader(to_instance)
        - vocab.from_instances
        - token_indexer(index_with => token_id)
        - dataloader(call batch_tensors in fields)
        - embedder(token_id2vec)
        - encoder(model part)
    - trainer
        - trainer
        - tensorboard_writer
        - config file composition and jsonnet
    - general work flow conclusion
        - rawdata =reader=> instance (with fields)
        - instance =Vocab=> vocab (with namespaces)
        - instance =IndexWith=> indexed_instances (TensorDict)
        - instances =dataloader(batch_tensors function)=> batch_tensor (TensorDict)
        - batch_tensor =model=> logits
    - allennlp conclusion
        - OOP + dependency injection
        - a good coding style
        - several robust off-the-shelf models
- implementation of datareader
    - use utils.doc2graph
    - graph2instance
- implementation of SparseAdjacencyField
    - sparse version of origin AdjacencyField
    - modify code (almost all) of allennlp AdjacencyField
    - implementation of PytorchGeoData Batching

# Stage 2 - Train a naive model (BagofWordPooling) with allennlp train
- 8/?-8/?
- (start this note when 2->3)
- mismatched BERT (use defualt mean)
    - use PretrainedTransformerMismatchedIndexer + PretrainedTransformerMismatchedEmbedder
    - note that here use BERT without special token
    - also "[ROOT]" in dependency graph is not special token to BERT is a potential issue
- sparse2dense, dense2sparse in tensorop.py
    - naive implementation works well without tensor
    - fix gradient issue
        - learn about leaf node in computatino graph
        - inplace operation
        - tensor properties
        - torch.sparse.Tensor.to_dense() as tf.scatter_nd
    - 2020/8/21, can actually use pytorch_scatter, pytorch_sparse...
- allennlp train can work with my modules
    
    
# (Now) State 3 - Train A HGNN model (het graph embedding w/o interaction)
- due 8/22
- add Graph2VecEncoder Registrable
- implement HGEN

# Stage 4 - Train A HGMN model (het graph matching network (may be final))
- due 8/31
- add GraphPair2VecEncoder Registrable
- implement HGMN

# Stage 5 - Validation on ANLI/Q-Test/HAN, Experiments
- due 9/15
- parse ANLI/HAN
- Q-Test generator(This may be required earlier)

# Stage 6 - Paper Fixing (due 9/19, EACL due 9/20)

# Now Work, Todo
- reader to add bidirectional relation
    - add add_edge for simplicity
- GraphEMbeddingNet(GraphPair2VecEncoder)
    - todo
- Config is modified
    - remove or transformer embedder
    - can train on token embedding first (quicker and see effect)
    - also a must do exp
- add raw_text_datareader
- tensor_op
    - move sparse cross attention to tensor_op

In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
import config
import utils
import reader

In [21]:
bert_model = "bert-base-uncased"
train_data_path = "../data/anli_v1.0/R1/train.jsonl"
validation_data_path = "../data/anli_v1.0/R1/dev.jsonl"
test_data_path = "../data/anli_v1.0/R1/test.jsonl"
cache_data_dir = "../data/ANLI_instance_cache"

## Stanza NLP

In [13]:
import stanza
stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')

2020-08-22 18:03:36 INFO: Downloading default packages for language: en (English)...
2020-08-22 18:03:38 INFO: File exists: /root/stanza_resources/en/default.zip.
2020-08-22 18:03:44 INFO: Finished downloading models and saved to /root/stanza_resources.
2020-08-22 18:03:44 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-08-22 18:03:44 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| depparse  | ewt     |

2020-08-22 18:03:44 INFO: Use device: gpu
2020-08-22 18:03:44 INFO: Loading: tokenize
2020-08-22 18:03:48 INFO: Loading: pos
2020-08-22 18:03:50 INFO: Loading: lemma
2020-08-22 18:03:50 INFO: Loading: depparse
2020-08-22 18:03:51 INFO: Done loading processors!


In [23]:
rdr = reader.NLIGraphReader(input_parsed=False, parser=nlp, cache_directory=cache_data_dir, input_fields=reader.config.anli_fields)

In [24]:
rdr.read(file_path=validation_data_path)

KeyError: 'premise'